# Visualise latent space

The aim of this notebook is to replicate a figure from the lecture notes of _Cambridge Engineering-Deep Learning for Computer Vision_. The task is binary classification with a 3 node hidden layer. This hidden layer is visualised in 3D space, which showed that the action of learning _pushed_ the weights into corners of the cube.

In [1]:
# Relative import hack
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))

# Setup logging
import pickle
from pathlib import Path
import numpy as np
import seaborn as sns
import matplotlib.animation as ani
from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from digit_classifier import base, DigitClassifier, DEVICE, Metrics, DigitClassifierConfig, train_test_datasets, train_val_dataset_split
import logging

sns.set_theme()

[digit_classifier] [INFO] Added console handler
[digit_classifier] [INFO] Torch device: cpu


In [2]:
base() # Set seeds

In [3]:
# Model parameters
config = DigitClassifierConfig(
	sizes=[784, 10, 3, 10],
	learning_rate=1,
  device=DEVICE,
  loss = nn.MSELoss(reduction='mean'),
  mini_batch = 20,
)

model_dir = Path("../../resources/model/latent_space/")
metrics_dir = model_dir / 'metrics.pkl'

In [4]:
# Extract the labels of the training set
train_set = train_val_dataset_split(train_test_datasets()[0])[0]
labels = [train_set[i][1] for i in range(len(train_set))]
labels = np.array(labels)
labels.shape

[digit_classifier] [INFO] Train size: 60000
[digit_classifier] [INFO] Test size: 10000
[digit_classifier] [INFO] Train set: 50000
[digit_classifier] [INFO] Valid set: 10000


(50000,)

In [6]:
INSPECT_LAYER = 1  # Which layer to inspect. Set this to K - 1

# Variable to store the latent space output and classifier label
# Shape: (epoch, training example, latent space)
latent_label = []

latent_dataloader = torch.utils.data.DataLoader(train_set, batch_size=1000, shuffle=False,num_workers=2, drop_last=False)

@torch.no_grad()
def func(dc: DigitClassifier, **func_kwargs):
  """Keep track of the latent space output

  Args:
    dc: DigitClassifier
    ll: np array to store the outputs

  After each batch/epoch, evaluate on the training set (with no grad)
  for each data point
  - run a forward pass
  - keep track of the latent space, and the output
  """
  ll: list = func_kwargs['ll']

  dc.eval()
  # Increment ll axis 0
  ll.append([])
  # ll = np.append(ll, np.zeros((1, 1, config.sizes[1] + 1)), axis=0)
  for x, y in latent_dataloader:
    for layer_idx, layer in enumerate(dc.linears):
      x = layer(x)
      x = dc.act_fn(x)
      if layer_idx == INSPECT_LAYER:
        break
    latent_space = x.detach().numpy()
    # latent_space = np.append(latent_space, y.cpu().numpy()[:,None], axis=1)
    ll[-1] += latent_space.tolist()

## Train Model

In [7]:
model = DigitClassifier(config)
model

[digit_classifier] [INFO] Train size: 60000
[digit_classifier] [INFO] Test size: 10000
[digit_classifier] [INFO] Train set: 50000
[digit_classifier] [INFO] Valid set: 10000


DigitClassifier(
  (act_fn): Sigmoid()
  (linears): ModuleList(
    (0): Linear(in_features=784, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=3, bias=True)
    (2): Linear(in_features=3, out_features=10, bias=True)
  )
  (loss_module): MSELoss()
)

In [8]:
epochs = 50
metrics = Metrics()
model.train_loop(num_epochs=epochs, metrics=metrics, func=func, ll=latent_label)

[digit_classifier] [INFO] Epoch: 0: 1149 / 10000
[digit_classifier] [INFO] Epoch: 1: 2076 / 10000
[digit_classifier] [INFO] Epoch: 2: 2773 / 10000
[digit_classifier] [INFO] Epoch: 3: 4278 / 10000
[digit_classifier] [INFO] Epoch: 4: 4718 / 10000
[digit_classifier] [INFO] Epoch: 5: 5290 / 10000
[digit_classifier] [INFO] Epoch: 6: 5684 / 10000
[digit_classifier] [INFO] Epoch: 7: 5885 / 10000
[digit_classifier] [INFO] Epoch: 8: 5840 / 10000
[digit_classifier] [INFO] Epoch: 9: 5922 / 10000
[digit_classifier] [INFO] Epoch: 10: 6188 / 10000
[digit_classifier] [INFO] Epoch: 11: 6262 / 10000
[digit_classifier] [INFO] Epoch: 12: 6550 / 10000
[digit_classifier] [INFO] Epoch: 13: 6744 / 10000
[digit_classifier] [INFO] Epoch: 14: 7326 / 10000
[digit_classifier] [INFO] Epoch: 15: 7598 / 10000
[digit_classifier] [INFO] Epoch: 16: 7733 / 10000
[digit_classifier] [INFO] Epoch: 17: 8041 / 10000
[digit_classifier] [INFO] Epoch: 18: 8187 / 10000
[digit_classifier] [INFO] Epoch: 19: 8243 / 10000
[digit_cla

In [9]:
# Save model
model.save_model(model_dir)

(PosixPath('../../resources/model/latent_space/model.tar'),
 PosixPath('../../resources/model/latent_space/config.pkl'))

In [10]:
latent_label = np.array(latent_label)
latent_label.shape

(50, 50000, 3)

In [11]:
# Convert to numpy array and save
f = model_dir / f'latents-{epochs}.npy'
np.save(f, latent_label)
f = model_dir / f'labels.npy'
np.save(f, labels)